# 필요한 라이브러리 import

In [10]:
import pandas as pd
import os
import re
import csv
import joblib
import warnings
import numpy as np

# 사전작업

In [11]:
warnings.filterwarnings(action='ignore')

# 포지션 판정 모델 위치
model_path = './models/position_model.pkl'

# 초기 데이터 위치(포지션 판정)
data_path = './datas/NEW_FM_DATA'

# 변환된 데이터 위치
target_path = './datas/NEW_FM_DATA_MERGED'

# 선형회귀를 위한 데이터셋을 저장할 위치
ml_data_path = './datas/NEW_FM_DATA_FOR_ML'

# 포지션 판정 모델 불러오기
position_predict_model=joblib.load(model_path)

# 데이터 전처리

## 한 시즌의 모든 팀의 데이터를 병합하는 함수

In [12]:
def data_merge(path,season):
    
    # 최종 데이터 프레임 컬럼
    var_list = ['이름', '포지션', '구단', '리그', '임대 기간', '승', '무', '패', '출장시간',
       '경기당 팀 득점/90', '경기당 팀 실점/90', '골', '도움', 'Cr C/A', '헤더 시도', 'Cr A',
       'FA', 'Fls', '경기 당 드리블', '드리블', '기회 창출/90', '경기당 유효 슈팅/90', '경기당 슈팅/90',
       'Int/90', '달린 거리/90분', '공중 A/90', '경기당 헤더 성공', '태클 성공률', '태클',
       '경기당 패스 시도/90', '경기당 패스 성공/90', 'K Ps/90','시즌']
    
    # 모든 변수를 가진 빈 데이터 프레임
    team_df_whole_league = pd.DataFrame(columns=var_list)

    # 특정 경로의 모든 파일 리스트 불러오기(특정리그의 모든 팀들이 저장되어 있음)
    file_list = os.listdir(path)
    
    # 모든 파일을 돌며 -> 각 팀의 통계 자료를 team_df_whole_league 에 합침
    for file in file_list:

        #불러오기
        team_df = pd.read_csv(path+'/'+file,encoding='utf-8')
        
        # 결측치 제거 제거
        team_df = team_df.dropna()
    
        # 골키퍼는 통계 자료 부족으로 삭제
        team_df = team_df[team_df['포지션'] != 'GK']

        # 현재 구단에 소속되었는지 여부를 판단하기 위해 현재 구단의 이름을 추출
        team = team_df[team_df['임대 기간']=='-']['구단'].iloc[0]

        # '-'을 0으로 교체
        team_df = team_df.replace('-',0)

        # 현재 구단에 속해 있으며 출장 시간이 900분을 넘긴 선수만 남김
        team_df = team_df[(team_df['구단']==team) & (team_df['출장시간'].astype('int64')>900)]

        # 시즌 열을 추가
        team_df['시즌'] = season

        # team_df_whole_league 에 연결
        team_df_whole_league=pd.concat([team_df_whole_league,team_df])
        
    # team_df_whole_league의 인덱스를 재설정
    team_df_whole_league = team_df_whole_league.reset_index(drop=True)
    
    # nan은 0으로 대체
    team_df_whole_league = team_df_whole_league.fillna(0)
    
    return team_df_whole_league

## 선수 포지션을 추정하는 함수

In [13]:
def position_predictor(league_df):
    
    origin_df = league_df.copy()
    
    # 포지션 추정을 위한 컬럼
    columns_for_ml = ['경기 당 드리블', '기회 창출/90', '경기당 유효 슈팅/90',
       '경기당 슈팅/90', 'Int/90', '공중 A/90', '경기당 헤더 성공', '태클', '경기당 패스 시도/90',
       '경기당 패스 성공/90', 'K Ps/90']
    
    # 포지션 추정을 위한 df
    df_for_position_ml = league_df[columns_for_ml]
    
    # 판정 결과 저장(포지션 열)
    origin_df['포지션'] = position_predict_model.predict(df_for_position_ml)
    
    origin_df['포지션'] = origin_df['포지션'].map(position_label_reverse)
    
    return origin_df

## 포지션 숫자 -> 진짜 포지션

In [14]:
def position_label_reverse(x):
    if x == 0:
        return '미드필더'
    
    elif x == 1:
        return '수비수'
    
    else:
        return '공격수'


## 포지션 추정된 데이터 전처리 및 변환하는 함수

In [15]:
def data_cleaner(df):
    
    new_df = df.copy()
    
    # 단위 제거
    new_df['Cr C/A'] = new_df['Cr C/A'].str.strip('%')
    new_df['달린 거리/90분'] = new_df['달린 거리/90분'].str.strip('km')
    new_df['태클 성공률'] = new_df['태클 성공률'].str.strip('%')
    
    
    # 판정 후 추가 데이터 전처리 진행,형변환
    new_df = new_df.astype({'승':np.int,'무':np.int,'패':np.int,
                        '출장시간':np.int,
                        '경기당 팀 득점/90':np.float,'경기당 팀 실점/90':np.float,
                        '골':np.int,'도움':np.int,
                        'Cr C/A':np.float,'Cr A':np.float,
                        'FA':np.int,'Fls':np.int,
                        '경기 당 드리블':np.float,'드리블':np.int,
                        '기회 창출/90':np.float, 
                        '경기당 유효 슈팅/90':np.float,'경기당 슈팅/90':np.float,
                        'Int/90':np.float,'달린 거리/90분':np.float,
                        '공중 A/90':np.float,'경기당 헤더 성공':np.float,
                        '태클 성공률':np.int,'태클':np.float,
                        '경기당 패스 시도/90':np.float,'경기당 패스 성공/90':np.float,
                        'K Ps/90':np.float,
                        '헤더 시도':np.int})
    
    # 90분당 횟수로 변환하는 이유는? 선수마다 출전시간이 다 다르기 때문에 많이 출전한 선수가 상대적으로 잘한다고 오류를 낼 수 있다.
    # 또한 경기당 피쳐의 횟수도 90분 단위로 변환시켜 주어야 한다. 왜냐하면 경기출전 = 90분 출전이 아니기 때문이다.
    
    # 경기당 태클 횟수를 90분당 태클 횟수로 변환
    new_df['태클 성공/90'] = np.round((new_df['승']+new_df['무']+new_df['패'])*new_df['태클']/new_df['출장시간']*90,2)
    new_df = new_df.astype({'태클 성공/90':np.float16})
    
    # 시즌 드리블 성공횟수를 90분당 드리블 성공으로 변환
    new_df['드리블 성공/90'] = np.round(new_df['드리블']/new_df['출장시간']*90,2)
    new_df = new_df.astype({'드리블 성공/90':np.float16})
    
    # 경기당 헤더 성공을 90분당 헤더 성공 횟수로 변환
    new_df['헤더 성공/90'] = np.round((new_df['승']+new_df['무']+new_df['패'])*new_df['경기당 헤더 성공']/new_df['출장시간']*90,2)
    new_df = new_df.astype({'헤더 성공/90':np.float16})
    
    # 시즌 크로스 시도를 90분당 크로스 시도로 변환
    new_df['Cr A/90'] = np.round(new_df['Cr A']/new_df['출장시간']*90,2)
    new_df = new_df.astype({'Cr A/90':np.float16})
    
    # 시즌 반칙 횟수를 90분당 반칙횟수로 변환
    new_df['반칙 수/90'] = np.round(new_df['Fls']/new_df['출장시간']*90,2)
    new_df = new_df.astype({'반칙 수/90':np.float16})
    
    # 변환 전 컬럼을 삭제
    new_df = new_df.drop(['태클','경기당 헤더 성공','Cr A','드리블','경기 당 드리블','헤더 시도','Fls'],axis=1)

    # 승점 또한 900분으로 표준화
    new_df['평균획득승점/경기'] = np.round((new_df['승']*3 + new_df['무'])/(new_df['승']+ new_df['무']+new_df['패']),2)
    
    return new_df

# 시즌 별, 리그 단위로 데이터 병합 및 저장

In [16]:
# 시즌을 추출할 정규표현식(숫자)
season_re = re.compile('\d+')

# 리그를 추출할 정규표현식
league_re = re.compile('[a-zA-Z]+')

# NEW_FM_DATA 내의 목록 가져오기
folder_list = os.listdir(data_path)

In [17]:
# 매 폴더를 돌며
for folder in folder_list:

    # 상위경로와 하위 경로를 병합할 변수
    sub_path = ''

    # 폴더명으로 새 경로를 생성
    sub_path = os.path.join(data_path,folder)
    sub_folder_list = os.listdir(sub_path)
    
    for sub_folder in sub_folder_list:
        
        sub_sub_path = os.path.join(data_path,folder,sub_folder)

        # 시즌을 추출
        season = season_re.findall(folder)

        # 리그를 추출
        league = league_re.findall(sub_folder)

        # 2020-2021의 형태로 시즌을 만들기
        season = season[0] + '-' + season[1]

        # 한 시즌의 모든 팀들의 데이터를 병합하고 전처리
        df_merged = data_merge(sub_sub_path, season)
    
        # 포지션 모델을 이용해서 선수들의 포지션을 판정
        df_with_position = position_predictor(df_merged)
        
        # 포지션 판정된 데이터 전처리 및 변환
        df_clean = data_cleaner(df_with_position)
        
        # 폴더명과 2020-2021-Ligue1.csv의 형식으로 파일을 저장
        os.makedirs(target_path+'/'+season,exist_ok=True)
        csv_path = target_path+'/'+season+'/'+ season +'-'+ league[0]+ '.csv'
        csv_file = open(csv_path,'w',newline='',encoding='utf-8-sig')
        csvwriter = csv.writer(csv_file)
        csvwriter.writerow(['이름' , '시즌','포지션', '구단', '리그' , '골', '도움',
                            '임대 기간', '승', '무', '패', '출장시간', '경기당 팀 득점/90',
                            '경기당 팀 실점/90', 'Cr C/A','FA', '반칙 수/90', '기회 창출/90',
                            '경기당 유효 슈팅/90', '경기당 슈팅/90', 'Int/90', '달린 거리/90분', 
                            '공중 A/90', '태클 성공률','경기당 패스 시도/90', '경기당 패스 성공/90', 
                            'K Ps/90','태클 성공/90', '드리블 성공/90', 
                            '헤더 성공/90', 'Cr A/90', '평균획득승점/경기'])
        
        for i in range(len(df_clean)):
            csvwriter.writerow(df_clean.loc[i,['이름','시즌','포지션', '구단', '리그' , '골', '도움',
                            '임대 기간', '승', '무', '패', '출장시간', '경기당 팀 득점/90',
                            '경기당 팀 실점/90', 'Cr C/A', 'FA', '반칙 수/90', '기회 창출/90',
                            '경기당 유효 슈팅/90', '경기당 슈팅/90', 'Int/90', '달린 거리/90분', 
                            '공중 A/90', '태클 성공률','경기당 패스 시도/90', '경기당 패스 성공/90', 
                            'K Ps/90','태클 성공/90', '드리블 성공/90', 
                            '헤더 성공/90', 'Cr A/90', '평균획득승점/경기']].tolist())
        csv_file.close()
        
    print(f'{season} 완료.')

2021-2022 완료.
2022-2023 완료.
2023-2024 완료.


# 시즌별 데이터 병합

In [18]:
# 시즌 별 폴더리스트
season_dir = os.listdir(target_path)

# 컬럼명 정의

col_names = ['이름','승','무','패','시즌','리그','포지션','구단','골', '도움', '경기당 팀 득점/90',
                        '경기당 팀 실점/90', 'Cr C/A', 'FA', '반칙 수/90', '기회 창출/90',
                        '경기당 유효 슈팅/90', '경기당 슈팅/90', 'Int/90', '달린 거리/90분',
                        '공중 A/90', '태클 성공률','경기당 패스 시도/90', '경기당 패스 성공/90', 
                        'K Ps/90', '태클 성공/90', '드리블 성공/90', 
                        '헤더 성공/90', 'Cr A/90', '평균획득승점/경기']


# 모든 시즌 폴더를 순회하며
for this_dir in season_dir:
    
    # 특정 시즌 폴더
    sub_path = os.path.join(target_path,this_dir)
    
    # 특정 시즌 안의 리그파일의 경로들 
    sub_path_list = os.listdir(os.path.join(target_path,this_dir))
    
    # 빈 데이터 프레임을 생성
    df_for_ML = pd.DataFrame(columns = col_names)
    
    # 리그파일마다 순화하며
    for file in sub_path_list:
        
        # 머신러닝에 적용할 데이터프레임에 연결
        data_path = os.path.join(sub_path,file)
        this_df = pd.read_csv(data_path,encoding='utf-8-sig')
        df_for_ML = pd.concat([df_for_ML,this_df])
    
    # 인덱스 초기화
    df_for_ML = df_for_ML.reset_index(drop=True)
    
    # 시즌 추출
    season = season_re.findall(file)
    season = season[0] + '-' + season[1]
    
    # 병합된 시즌별 데이터를 저장
    os.makedirs(ml_data_path+'/'+season,exist_ok=True)
    csv_path = ml_data_path+'/'+season+'/'+ season +'ML'+ '.csv'
    csv_file = open(csv_path,'w',newline='',encoding='utf-8-sig')
    csvwriter = csv.writer(csv_file)
    
    csvwriter.writerow(col_names)
        
    for i in range(len(df_for_ML)):
        csvwriter.writerow(df_for_ML.loc[i,col_names].values)